In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch import nn 
import copy

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from time import time
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, confusion_matrix, precision_score, recall_score, auc
from sklearn.model_selection import KFold
torch.manual_seed(1)    # reproducible torch:2 np:3
np.random.seed(1)

from config import BIN_config_DBPE
from models import BIN_Interaction_Flat, BIN_Transformer_Single
from stream import BIN_Data_Encoder, BIN_combined_encoder

use_cuda = torch.cuda.is_available()
print(use_cuda)
device = torch.device("cuda:0" if use_cuda else "cpu")

False


In [2]:
import sys
print(sys.executable)

/Users/zdx_macos/.pyenv/versions/meta-rl/bin/python3


In [3]:
def test(data_generator, model):
    y_pred = []
    y_label = []
    model.eval()
    loss_accumulate = 0.0
    count = 0.0
#     for i, (d, p, d_mask, p_mask, label) in enumerate(data_generator):
# #         score = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())
#         score = model(d.long(), p.long(), d_mask.long(), p_mask.long())
#         m = torch.nn.Sigmoid()
#         logits = torch.squeeze(m(score))
        
#         loss_fct = torch.nn.BCELoss()            
        
# #         label = Variable(torch.from_numpy(np.array(label)).float()).cuda()
#         label = Variable(torch.from_numpy(np.array(label)).float())

#         loss = loss_fct(logits, label)
        
#         loss_accumulate += loss
#         count += 1
        
#         logits = logits.detach().cpu().numpy()
        
#         label_ids = label.to('cpu').numpy()
#         y_label = y_label + label_ids.flatten().tolist()
#         y_pred = y_pred + logits.flatten().tolist()
        
    for i, (feature, mask, label) in enumerate(data_generator):
#         score = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())
        score = model(feature.long(), mask.long())
        m = torch.nn.Sigmoid()
        logits = torch.squeeze(m(score))
        
        loss_fct = torch.nn.BCELoss()            
        
#         label = Variable(torch.from_numpy(np.array(label)).float()).cuda()
        label = Variable(torch.from_numpy(np.array(label)).float())

        loss = loss_fct(logits, label)
        
        loss_accumulate += loss
        count += 1
        
        logits = logits.detach().cpu().numpy()
        
        label_ids = label.to('cpu').numpy()
        y_label = y_label + label_ids.flatten().tolist()
        y_pred = y_pred + logits.flatten().tolist()
        
    loss = loss_accumulate/count
    
    fpr, tpr, thresholds = roc_curve(y_label, y_pred)

    precision = tpr / (tpr + fpr)

    f1 = 2 * precision * tpr / (tpr + precision + 0.00001)

    thred_optim = thresholds[5:][np.argmax(f1[5:])]

    print("optimal threshold: " + str(thred_optim))

    y_pred_s = [1 if i else 0 for i in (y_pred >= thred_optim)]

    auc_k = auc(fpr, tpr)
    print("AUROC:" + str(auc_k))
    print("AUPRC: "+ str(average_precision_score(y_label, y_pred)))

    cm1 = confusion_matrix(y_label, y_pred_s)
    print('Confusion Matrix : \n', cm1)
    print('Recall : ', recall_score(y_label, y_pred_s))
    print('Precision : ', precision_score(y_label, y_pred_s))

    total1=sum(sum(cm1))
    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    print('Sensitivity : ', sensitivity1 )

    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    print('Specificity : ', specificity1)

    outputs = np.asarray([1 if i else 0 for i in (np.asarray(y_pred) >= 0.5)])
    return roc_auc_score(y_label, y_pred), average_precision_score(y_label, y_pred), f1_score(y_label, outputs), y_pred, loss.item()


def main(fold_n, lr):
    config = BIN_config_DBPE()
    
    lr = lr
    BATCH_SIZE = config['batch_size']
    train_epoch = 10
    
    loss_history = []
    
#     model = BIN_Interaction_Flat(**config)
    model = BIN_Transformer_Single(**config)
    
    if use_cuda:
        model = model.cuda()

    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model, dim = 0)
            
    opt = torch.optim.Adam(model.parameters(), lr = lr)
    #opt = torch.optim.SGD(model.parameters(), lr = lr, momentum=0.9)
    
    print('--- Data Preparation ---')
    
    params = {'batch_size': BATCH_SIZE,
              'shuffle': True,
              'num_workers': 6, 
              'drop_last': True}

    dataFolder = './dataset/BindingDB'
    df_train = pd.read_csv(dataFolder + '/train.csv')
    df_val = pd.read_csv(dataFolder + '/val.csv')
    df_test = pd.read_csv(dataFolder + '/test.csv')
    
#     training_set = BIN_Data_Encoder(df_train.index.values, df_train.Label.values, df_train)
#     training_generator = data.DataLoader(training_set, **params)

#     validation_set = BIN_Data_Encoder(df_val.index.values, df_val.Label.values, df_val)
#     validation_generator = data.DataLoader(validation_set, **params)
    
#     testing_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, df_test)
#     testing_generator = data.DataLoader(testing_set, **params)
    
    train_set = BIN_combined_encoder(df_train.index.values, df_train.Label.values, df_train)
    train_gen = data.DataLoader(train_set, **params)
    
    valid_set = BIN_combined_encoder(df_val.index.values, df_val.Label.values, df_val)
    valid_gen = data.DataLoader(valid_set, **params)
    
    test_set = BIN_combined_encoder(df_test.index.values, df_test.Label.values, df_test)
    test_gen = data.DataLoader(test_set, **params)
    # early stopping
    max_auc = 0
    model_max = copy.deepcopy(model)
    
    print('--- Go for Training ---')
    torch.backends.cudnn.benchmark = True
    for epo in range(train_epoch):
        model.train()
        print(len(train_gen))
        for i, (feature, mask, label) in enumerate(train_gen):
            score = model(feature.long(), mask.long())
            label = Variable(torch.from_numpy(np.array(label)).float())
            loss_fct = torch.nn.BCELoss()
            m = torch.nn.Sigmoid()
            n = torch.squeeze(m(score))
            
            loss = loss_fct(n, label)
            loss_history.append(loss)
            
            opt.zero_grad()
            loss.backward()
            opt.step()
            
            if (i % 100 == 0):
                print('Training at Epoch ' + str(epo + 1) + ' iteration ' + str(i) + ' with loss ' + str(loss.cpu().detach().numpy()))
            if (i % 1000 == 0):
                # every epoch test
                with torch.set_grad_enabled(False):
                    auc, auprc, f1, logits, loss = test(valid_gen, model)
                    if auc > max_auc:
                        model_max = copy.deepcopy(model)
                        max_auc = auc
            
                print('Validation at Epoch '+ str(epo + 1) + ' , AUROC: '+ str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: '+str(f1))
            
#         for i, (d, p, d_mask, p_mask, label) in enumerate(training_generator):
# #             score = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())
#             score = model(d.long(), p.long(), d_mask.long(), p_mask.long())
# #             label = Variable(torch.from_numpy(np.array(label)).float()).cuda()
#             label = Variable(torch.from_numpy(np.array(label)).float())
#             loss_fct = torch.nn.BCELoss()
#             m = torch.nn.Sigmoid()
#             n = torch.squeeze(m(score))
            
#             loss = loss_fct(n, label)
#             loss_history.append(loss)
            
#             opt.zero_grad()
#             loss.backward()
#             opt.step()
            
#             if (i % 100 == 0):
#                 print('Training at Epoch ' + str(epo + 1) + ' iteration ' + str(i) + ' with loss ' + str(loss.cpu().detach().numpy()))
            # every epoch test
        with torch.set_grad_enabled(False):
            auc, auprc, f1, logits, loss = test(valid_gen, model)
            if auc > max_auc:
                model_max = copy.deepcopy(model)
                max_auc = auc
        
    print('--- Go for Testing ---')
    try:
        with torch.set_grad_enabled(False):
            auc, auprc, f1, logits, loss = test(test_gen, model_max)
            print('Testing AUROC: ' + str(auc) + ' , AUPRC: ' + str(auprc) + ' , F1: '+str(f1) + ' , Test loss: '+str(loss))
    except:
        print('testing failed')
    return model_max, loss_history

In [ ]:
# fold 1
#biosnap interaction times 1e-6, flat, batch size 64, len 205, channel 3, epoch 50
s = time()
model_max, loss_history = main(1, 5e-6)
e = time()
print(e-s)
lh = list(filter(lambda x: x < 1, loss_history))
plt.plot(lh)

--- Data Preparation ---
--- Go for Training ---
3167
Training at Epoch 1 iteration 0 with loss 0.61577725


/var/folders/g0/93h1vwd10rq00w5yx9hftvxm0000gq/T/ipykernel_74238/2916935592.py:55: RuntimeWarning: invalid value encountered in true_divide
  precision = tpr / (tpr + fpr)


optimal threshold: 0.4846193492412567
AUROC:0.4983798957631048
AUPRC: 0.13455971626822696
Confusion Matrix : 
 [[ 110 5607]
 [   5  922]]
Recall :  0.9946062567421791
Precision :  0.1412161127278297
Accuracy :  0.15532811559301626
Sensitivity :  0.01924086059121917
Specificity :  0.9946062567421791
Validation at Epoch 1 , AUROC: 0.4983798957631048 , AUPRC: 0.13455971626822696 , F1: 0.06906077348066299
Training at Epoch 1 iteration 100 with loss 0.69492555
Training at Epoch 1 iteration 200 with loss 0.7137166
Training at Epoch 1 iteration 300 with loss 0.67332923
Training at Epoch 1 iteration 400 with loss 0.606265
Training at Epoch 1 iteration 500 with loss 0.6076871
Training at Epoch 1 iteration 600 with loss 0.54651904
Training at Epoch 1 iteration 700 with loss 0.7958389
Training at Epoch 1 iteration 800 with loss 0.7409822
Training at Epoch 1 iteration 900 with loss 0.735723
Training at Epoch 1 iteration 1000 with loss 0.7000242


/var/folders/g0/93h1vwd10rq00w5yx9hftvxm0000gq/T/ipykernel_74238/2916935592.py:55: RuntimeWarning: invalid value encountered in true_divide
  precision = tpr / (tpr + fpr)


optimal threshold: 0.21927951276302338
AUROC:0.8113720146900019
AUPRC: 0.40630383336549664
Confusion Matrix : 
 [[3100 2617]
 [ 108  819]]
Recall :  0.883495145631068
Precision :  0.23835855646100115
Accuracy :  0.589855508729681
Sensitivity :  0.5422424348434494
Specificity :  0.883495145631068
Validation at Epoch 1 , AUROC: 0.8113720146900019 , AUPRC: 0.40630383336549664 , F1: 0.33216538192011213
Training at Epoch 1 iteration 1100 with loss 0.3320628
Training at Epoch 1 iteration 1200 with loss 0.497371
Training at Epoch 1 iteration 1300 with loss 0.9707896
Training at Epoch 1 iteration 1400 with loss 0.35332763
Training at Epoch 1 iteration 1500 with loss 0.69269955
Training at Epoch 1 iteration 1600 with loss 0.5850935
Training at Epoch 1 iteration 1700 with loss 0.43192643
Training at Epoch 1 iteration 1800 with loss 0.7938974
Training at Epoch 1 iteration 1900 with loss 0.24118125
Training at Epoch 1 iteration 2000 with loss 0.42056596


...

Training at Epoch 7 iteration 0 with loss 0.65147936


/var/folders/g0/93h1vwd10rq00w5yx9hftvxm0000gq/T/ipykernel_74238/2916935592.py:55: RuntimeWarning: invalid value encountered in true_divide
  precision = tpr / (tpr + fpr)


optimal threshold: 0.4399678409099579
AUROC:0.8811787701812512
AUPRC: 0.5392874262979078
Confusion Matrix : 
 [[4237 1480]
 [ 116  811]]
Recall :  0.8748651564185544
Precision :  0.3539938891313837
Accuracy :  0.7597832630945214
Sensitivity :  0.7411229665908693
Specificity :  0.8748651564185544
Validation at Epoch 7 , AUROC: 0.8811787701812512 , AUPRC: 0.5392874262979078 , F1: 0.5208053691275167
Training at Epoch 7 iteration 100 with loss 0.9035216
Training at Epoch 7 iteration 200 with loss 0.37151074
Training at Epoch 7 iteration 300 with loss 0.65344715
Training at Epoch 7 iteration 400 with loss 0.37424666
Training at Epoch 7 iteration 500 with loss 0.118147954
Training at Epoch 7 iteration 600 with loss 0.31557864
Training at Epoch 7 iteration 700 with loss 0.11143083
Training at Epoch 7 iteration 800 with loss 0.519423
Training at Epoch 7 iteration 900 with loss 0.3015175
Training at Epoch 7 iteration 1000 with loss 0.22564462


/var/folders/g0/93h1vwd10rq00w5yx9hftvxm0000gq/T/ipykernel_74238/2916935592.py:55: RuntimeWarning: invalid value encountered in true_divide
  precision = tpr / (tpr + fpr)


optimal threshold: 0.30973008275032043
AUROC:0.8828647277117263
AUPRC: 0.5363769827367689
Confusion Matrix : 
 [[4014 1703]
 [  80  847]]
Recall :  0.9137001078748651
Precision :  0.33215686274509804
Accuracy :  0.731637567730283
Sensitivity :  0.7021164946650341
Specificity :  0.9137001078748651
Validation at Epoch 7 , AUROC: 0.8828647277117263 , AUPRC: 0.5363769827367689 , F1: 0.5301632363378282
Training at Epoch 7 iteration 1100 with loss 0.23497975
Training at Epoch 7 iteration 1200 with loss 0.7589859
Training at Epoch 7 iteration 1300 with loss 0.19997072
Training at Epoch 7 iteration 1400 with loss 1.1100525
Training at Epoch 7 iteration 1500 with loss 0.30895907
Training at Epoch 7 iteration 1600 with loss 0.2906143
Training at Epoch 7 iteration 1700 with loss 0.27142146


In [ ]:
torch.tensor([1.0, 2.0]).cuda()

In [4]:
print(torch.__version__)

1.11.0.dev20211106+cu113


In [23]:
config = BIN_config_DBPE()

BATCH_SIZE = config['batch_size']

params = {'batch_size': BATCH_SIZE,
              'shuffle': True,
              'num_workers': 6, 
              'drop_last': True}

dataFolder = './dataset/BIOSNAP/full_data'
df_train = pd.read_csv(dataFolder + '/train.csv')
df_val = pd.read_csv(dataFolder + '/val.csv')
df_test = pd.read_csv(dataFolder + '/test.csv')

training_set = BIN_Data_Encoder(df_train.index.values, df_train.Label.values, df_train)
training_generator = data.DataLoader(training_set, **params)

In [24]:
df_train.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,DrugBank ID,Gene,Label,SMILES,Target Sequence
0,0,3,4,DB08533,P49862,0.0,CC1=CN=C2N1C=CN=C2NCC1=CC=NC=C1,MARSLLLPLQILLLSLALETAGEEAQGDKIIDGAPCARGSHPWQVA...
1,1,4,5,DB00755,P48443,1.0,C\C(\C=C\C1=C(C)CCCC1(C)C)=C/C=C/C(/C)=C/C(O)=O,MYGNYSHFMKFPAGYGGSPGHTGSTSMSPSAALSTGKPMDSHPSYT...
2,2,5,6,DB00361,O60218,0.0,[H][C@@]12N(C)C3=CC(OC)=C(C=C3[C@@]11CCN3CC=C[...,MATFVELSTKAKMPIVGLGTWKSPLGKVKEAVKVAIDAGYRHIDCA...
3,3,7,8,DB01136,P08588,1.0,COC1=CC=CC=C1OCCNCC(O)COC1=CC=CC2=C1C1=CC=CC=C1N2,MGAGVLVLGASEPGNLSSAAPLPDGAATAARLLVPASPPASLLPPA...
4,4,8,9,DB06963,Q9Y691,0.0,[H][C@@](C)(NC1=CC2=C(C=N1)C(C)=NN2C1=CC=CC(CC...,MFIWTSGRTSSSYRHDEKRNIYQKIRDHDLLDKRKTVTALKAGEDR...


In [25]:
df_train["drug_encoding"][0]

KeyError: 'drug_encoding'

In [26]:
for i, (d, p, d_mask, p_mask, label) in enumerate(training_generator):
    print("DRUG:")
    print(d.shape)
    print(d)
    print("PROTEIN:")
    print(p.shape)
    print(p)
    print("DRUG MASK:")
    print(d_mask)
    print("PROTEIN MASK:")
    print(p_mask)
    print("LABEL")
    print(label.shape)
    print(label)
    break

DRUG:
torch.Size([16, 50])
tensor([[ 3291,   300,   471,   117,  7522,  1804,   147,  2268,    91,  1855,
            82,   211,   623,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    7,    66,  5763,    63,  3821, 13901,     7,    66,    11,  3901,
             7,    66,   189,  5374,   156,    10,     7,    66,    11,  1077,
          1176,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [10440, 18363,  6525, 20141,   569,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0, 